# Práctica 2: Procesamiento del Lenguaje Natural

__Fecha de entrega: 8 de mayo de 2023__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de PLN. La práctica consta de 2 notebooks que se entregarán simultáneamente en la tarea de entrega habilitada en el Campus  Virtual.

Lo más importante en esta práctica no es el código Python, sino el análisis de los datos y modelos que construyas y las explicaciones razonadas de cada una de las decisiones que tomes. __No se valorarán trozos de código o gráficas sin ningún tipo de contexto o explicación__.

Finalmente, recuerda establecer el parámetro `random_state` en todas las funciones que tomen decisiones aleatorias para que los resultados sean reproducibles (los resultados no varíen entre ejecuciones).

In [2]:
RANDOM_STATE = 1234

# Apartado 1: Análisis de sentimientos con word embeddings


__Número de grupo: XX__

__Nombres de los estudiantes: Gonzalo Garcia Fernández y Daniel María Carreño

## 1) Carga del conjunto de datos

El fichero `IMBD_Dataset.csv` contiene opiniones de películas clasificadas en 2 categorías diferentes (positiva/negativa).

Este set de datos se creó utilizando el "IMDB Dataset of 50K Movie Reviews", el cual contiene 50,000 reseñas de películas con un sentimiento positivo o negativo adjunto a ellas.

Muestra un ejemplo de cada clase.

Haz un estudio del conjunto de datos. ¿qué palabras aparecen más veces?, ¿tendría sentido normalizar de alguna manera el corpus?

Crea una partición de los datos dejando el 80% para entrenamiento y el 20% restante para test usando la función `train_test_split` de sklearn.


In [3]:
# acceso a google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import nltk
import re
import sklearn

In [5]:
imbd_file = '/content/drive/MyDrive/IMDB_Dataset.csv'

df=pd.read_csv(imbd_file)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

Vemos con el output del anterior que nuestra base de datos esta perfectamente equilibrada y tiene 50000 ejemplos(250000 negativos y 25000 positivos).

In [7]:

corpus =np.array(df['review'])
df2 = df[df['sentiment'] == 'positive']
print('Ejemplo positivo:')
print(df2.head(1)['review'])

df3 = df[df['sentiment'] == 'negative']
print('Ejemplo negativo:')
print(df3.head(1)['review'])


Ejemplo positivo:
0    One of the other reviewers has mentioned that ...
Name: review, dtype: object
Ejemplo negativo:
3    Basically there's a family where a little boy ...
Name: review, dtype: object


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

corpus =np.array(df['review'])
cv = CountVectorizer(max_features = 3)
cv_matrix = cv.fit_transform(corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[ 6,  7, 16],
       [ 7,  5, 16],
       [ 4,  4,  8],
       ...,
       [ 6, 11, 21],
       [ 8,  7, 13],
       [ 2,  2, 12]])

In [9]:
# get all unique words in the corpus
vocab = cv.get_feature_names_out()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

,and,of,the
0,6,7,16
1,7,5,16
2,4,4,8
3,4,2,6
4,5,6,20
...,...,...,...
49995,7,2,5
49996,2,4,9
49997,6,11,21
49998,8,7,13


Como podemos ver las palabras más tipicas del texto sin normalizar son las más utilizadas en inglés lo que no nos da ninguna información. Por lo tanto, normalizaremos el texto para poder ver realmente cuales son las palabras más utilizadas relevantes.

In [10]:
wpt = nltk.WordPunctTokenizer()
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = re.sub('br','',doc)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
norm_corpus = normalize_corpus(corpus)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 3)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 0, 0],
       [0, 0, 4],
       [0, 7, 1]])

In [13]:
# get all unique words in the corpus
vocab = cv.get_feature_names_out()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

,film,movie,one
0,0,0,1
1,0,0,1
2,0,0,1
3,2,3,0
4,2,1,6
...,...,...,...
49995,0,5,1
49996,0,0,0
49997,0,0,0
49998,0,0,4


Ahora ya podemos ver que nuestro dataset sera referente a peliculas porque las palabras más frecuentes son "movie" y "film".

In [14]:
from sklearn.model_selection import train_test_split 
review_train,review_test,sentiment_train,sentiment_test = train_test_split(df['review'],df['sentiment'],test_size= 0.20,stratify=df['sentiment'],random_state = RANDOM_STATE)#usamos la funcion train_test_split de sklear con un 20% de ejemplos para el test.
sentiment_train.value_counts()#contamos el numero de valoraciones positivas y negativas que hay en el train y vemos que son iguales



negative    20000
positive    20000
Name: sentiment, dtype: int64

In [15]:
sentiment_test.value_counts() #contamos el numero de valoraciones positivas y negativas que hay en el test y vemos que son iguales.

negative    5000
positive    5000
Name: sentiment, dtype: int64

Despues de realizar esto vemos que de los 50000 ejemplos nos ha cogido 40000 para el entrenamiento y 10000 para el test. Como podemos apreciar la distribucion de las clases está muy equilibrada gracias al parámetro añadido startify


## 2) Estudio del efecto de distintas configuraciones de word embeddings para resolver la tara

Usa distintas configuraciones de word embeddigns y discute los resultados obtenidos.



Vamos a entrenar el clasificador con 3 configuraciones de word embeddings distintas.

La primera de ella será sin entrenamiento previo, mientras que las otras dos harán uso de los embeddings pre-entrenados de 'glove.6B.50d.txt'. Una de ellas "congelará" lo ya pre-aprendido mientras que la otra podrá modificar esas capas.

Utilizamos la librería keras tal y como en el tutorial. Para ello necesitamos que la columna sentiments tenga un valor númerico. Dado que ahora mismo dicha columna vale 'positive' o 'negative', la modificamos por la columna sentiments_num y le asignamos 1 o 0 segun el valor de sentiments.

In [16]:
df = pd.DataFrame(list(zip(norm_corpus, df['sentiment'])),columns=['review', 'sentiment'] )

In [17]:
df['sentiment_num'] = df['sentiment']
for i in range(0, df['sentiment'].size):
  if (df['sentiment'][i] == 'positive'): df['sentiment_num'][i] = (1);
  else: df['sentiment_num'][i] = 0;


In [18]:
df2 = df.loc[:, ['review', 'sentiment_num']]
df2.head()

,review,sentiment_num
0,one reviewers mentioned watching oz episode yo...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically theres family little boy jake thinks...,0
4,petter matteis love time money visually stunni...,1


In [19]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

max_words = 1500
max_comment_length = 20

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df2.review)

sequences = tokenizer.texts_to_sequences(df2.review)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
max_words = len(word_index)

data = pad_sequences(sequences, maxlen=max_comment_length)

Found 162162 unique tokens.


In [20]:
from sklearn.model_selection import train_test_split

d=df2.values

x_train, x_test, y_train, y_test = train_test_split(data, df2.sentiment_num, test_size=0.20, random_state=RANDOM_STATE, stratify = df2.sentiment_num)

embedding_dim = 50

In [21]:
x_train = np.asarray(x_train).astype('float32')
x_test = np.asarray(x_test).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

In [22]:
# MODELO 1. SIN EMBEDDINGS PRE-ENTRENADOS 

from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model1 = Sequential()
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs


model1.add(Embedding(max_words, embedding_dim, input_length=max_comment_length))
# After the Embedding layer, our activations have shape `(max_words, max_comment_length, embedding_dim)`.

# We flatten the 3D tensor of embeddings into a 2D tensor of shape `(max_words, max_comment_length * embedding_dim)`

model1.add(Flatten())

# We add the classifier on top
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

history = model1.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))

score1 = model1.evaluate(x_test, y_test)

print("Accuracy: %.2f%%" % (score1[1]*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 50)            8108100   
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 1)                 1001      
                                                                 
Total params: 8,109,101
Trainable params: 8,109,101
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1250/1250 [==============================] - 32s 20ms/step - loss: 0.4894 - accuracy: 0.7645 - val_loss: 0.4259 - val_accuracy: 0.8015
Epoch 2/20
1250/1250 [==============================] - 7s 6ms/step - loss: 0.3942 - accuracy: 0.8198 - val_loss: 0.4288 - val_accuracy: 0.7991
Epoch 3/20
1250/1250

In [23]:
import os
import numpy as np

glove_dir = '/content/drive/MyDrive/'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [24]:
embedding_dim = 50

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [25]:
# MODELO 2. EMBEDDINGS PRE-ENTRENADOS CONGELADOS

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model2 = Sequential()
model2.add(Embedding(max_words, embedding_dim, input_length=max_comment_length))
model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 50)            8108100   
                                                                 
 flatten_1 (Flatten)         (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 8,109,101
Trainable params: 8,109,101
Non-trainable params: 0
_________________________________________________________________


In [26]:
model2.layers[0].set_weights([embedding_matrix])
model2.layers[0].trainable = False

In [27]:
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model2.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test))

score2 = model2.evaluate(x_test, y_test)

Epoch 1/20
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6106 - accuracy: 0.6625 - val_loss: 0.5759 - val_accuracy: 0.6979
Epoch 2/20
1250/1250 [==============================] - 4s 3ms/step - loss: 0.5708 - accuracy: 0.7030 - val_loss: 0.5803 - val_accuracy: 0.6877
Epoch 3/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.5673 - accuracy: 0.7081 - val_loss: 0.5870 - val_accuracy: 0.6870
Epoch 4/20
1250/1250 [==============================] - 4s 3ms/step - loss: 0.5653 - accuracy: 0.7076 - val_loss: 0.5719 - val_accuracy: 0.7007
Epoch 5/20
1250/1250 [==============================] - 4s 3ms/step - loss: 0.5646 - accuracy: 0.7074 - val_loss: 0.5772 - val_accuracy: 0.6930
Epoch 6/20
1250/1250 [==============================] - 4s 4ms/step - loss: 0.5649 - accuracy: 0.7084 - val_loss: 0.5726 - val_accuracy: 0.6961
Epoch 7/20
1250/1250 [==============================] - 4s 3ms/step - loss: 0.5649 - accuracy: 0.7090 - val_loss: 0.5892 - val_accuracy:

In [28]:
# MODELO3. EMBEDDINGS PREENTRENADOS SIN CONGELAR

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model3 = Sequential()
model3.add(Embedding(max_words, embedding_dim, input_length=max_comment_length))
model3.add(Flatten())
model3.add(Dense(1, activation='sigmoid'))
model3.summary()

model3.layers[0].set_weights([embedding_matrix])
model3.layers[0].trainable = True

model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model3.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test))

score3 = model3.evaluate(x_test, y_test)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 50)            8108100   
                                                                 
 flatten_2 (Flatten)         (None, 1000)              0         
                                                                 
 dense_2 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 8,109,101
Trainable params: 8,109,101
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1250/1250 [==============================] - 20s 15ms/step - loss: 0.5441 - accuracy: 0.7181 - val_loss: 0.4581 - val_accuracy: 0.7801
Epoch 2/20
1250/1250 [==============================] - 8s 6ms/step - loss: 0.4247 - accuracy: 0.8027 - val_loss: 0.4384 - val_accuracy: 0.7957
Epoch 3/20
1250/12

In [29]:
print("Sin word embeddings pre-entrenados")
print("Accuracy: %.2f%%" % (score1[1]*100))
print("Con word embeddings pre-entrenados congelados")
print("Accuracy: %.2f%%" % (score2[1]*100))
print("Con word embeddings pre-entrenados sin congelar")
print("Accuracy: %.2f%%" % (score3[1]*100))

Sin word embeddings pre-entrenados
Accuracy: 72.98%
Con word embeddings pre-entrenados congelados
Accuracy: 70.10%
Con word embeddings pre-entrenados sin congelar
Accuracy: 73.15%


Tras entrenar al clasificador con los tres métodos vemos que el que utiliza embeddings pre-entrenados y los congela da peores resultados que los otros dos. Esto se debe a que la tarea es lo suficientemente específica como para que un pre-entrenamiento más general no sea tan eficiente como el entrenamiento específico. 

Adicionalmente, el hecho de que el método que no utiliza pre-entrenamiento funcione bien se debe a que la cantidad de muestras que tenemos para el entrenamiento es bastante grande, si fuera muy pequeño este método podría dar muy malos resultados.

Sin embargo, el que mejor funciona es el que utiliza pre-entrenamientos pero no los congela. Este método aprovecha tanto la información de los pre-entrenamientos como la de todas las muestras que disponemos y acaba dando un mejor resultado que los otros dos.

## 3) Análisis final

Analiza con detalle el mejor clasificador. Busca un ejemplo mal clasificado de cada clase, justifica el error ¿se te ocurre alguna forma de solucionarlo?

Compara los resultados obtenidos con y sin word embeddings


In [47]:
model3.evaluate(x_test[4:5], y_test[4:5])

print(y_test[4])
df['review'][4]

1/1 [==============================] - 0s 35ms/step - loss: 5.1577 - accuracy: 0.0000e+00
0.0


'petter matteis love time money visually stunning film watch mr mattei offers us vivid portrait human relations movie seems telling us money power success people different situations encounter variation arthur schnitzlers play theme director transfers action present time new york different characters meet connect one connected one way another next person one seems know previous point contact stylishly film sophisticated luxurious look taken see people live world live habitat thing one gets souls picture different stages loneliness one inhabits big city exactly best place human relations find sincere fulfillment one discerns case people encounter acting good mr matteis direction steve buscemi rosario dawson carol kane michael imperioli adrian grenier rest talented cast make characters come alive wish mr mattei good luck await anxiously next work'

In [57]:
np.array(model1.predict(x_test) >=0.5,dtype= int)[4]

313/313 [==============================] - 1s 2ms/step


array([1])

In [60]:
imbd_file = '/content/drive/MyDrive/IMDB_Dataset.csv'
dfo=pd.read_csv(imbd_file)

x = 8
i = 0
while i < len(sentiment_test) and x != 9:
  if (0 != np.array(model1.predict(x_test) >=0.5,dtype= int)[i][0] and dfo.sentiment[i]  == 'positive'):
    x = 9
  else:
    i = i +1 
print(i)
print("Resultado que debería dar: ",dfo.sentiment[i])
print("Resultado que da nuestro clasificador: negative")
print("Imprimimos el mensaje:\n")
print(dfo.review[i])

313/313 [==============================] - 0s 1ms/step
4
Resultado que debería dar:  positive
Resultado que da nuestro clasificador: negative
Imprimimos el mensaje:

Petter Mattei's "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler's play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. Stylishly, the film has a sophisticated luxurious look. We are taken to see how these people live and the world they live in their own habitat.<br /><br />The only thing one gets out of all these souls in the picture is the different 

En este ejemplo en el que muestra un resultado negativo cuando debería ser positivo, parece que el clasificador se equivoca debido a unas pocas palabras negativas como "loneliness".

In [64]:
imbd_file = '/content/drive/MyDrive/IMDB_Dataset.csv'
dfo=pd.read_csv(imbd_file)

x = 8
i = 0
while i < len(sentiment_test) and x != 9:
  if (0 != np.array(model1.predict(x_test) >=0.5,dtype= int)[i][0] and dfo.sentiment[i]  == 'negative'):
    x = 9
  else:
    i = i +1 
print(i)
print("Resultado que debería dar: ",dfo.sentiment[i])
print("Resultado que da nuestro clasificador: positive")
print("Imprimimos el mensaje:\n")
print(dfo.review[i])

313/313 [==============================] - 1s 2ms/step
11
Resultado que debería dar:  negative
Resultado que da nuestro clasificador: positive
Imprimimos el mensaje:

I saw this movie when I was about 12 when it came out. I recall the scariest scene was the big bird eating men dangling helplessly from parachutes right out of the air. The horror. The horror.<br /><br />As a young kid going to these cheesy B films on Saturday afternoons, I still was tired of the formula for these monster type movies that usually included the hero, a beautiful woman who might be the daughter of a professor and a happy resolution when the monster died in the end. I didn't care much for the romantic angle as a 12 year old and the predictable plots. I love them now for the unintentional humor.<br /><br />But, about a year or so later, I saw Psycho when it came out and I loved that the star, Janet Leigh, was bumped off early in the film. I sat up and took notice at that point. Since screenwriters are making u

En este ejemplo en el que le da una clasificación positiva cuando debería ser negativa es difícil ver la razón, aunque puede que sea porque otras reviews que hablan de ver la película de pequeños son más positivas.

Al comparar con los otros métodos sin word embeddings de la primera parte de la práctica, podemos ver que la precisión en test obtenida es muy similar a la que opbteníamos con árboles de decisión. Sin embargo, es peor que la que obteníamos con Naive-Bayes, por lo que este parece ser el mejor método para afrontar esta tarea, y no el de word embeddings.